## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [2]:
# load xlnet generated word embeddings
E = WordEmbedding('./embeddings/xlnet_specials.csv')

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/xlnet_specials.csv
(26423, 768)
26423 words of dimension 768 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
26423 words of dimension 768 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [3]:
# gender direction
v_gender = E.diff('she', 'he')

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [4]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, max_words=1000)

# for (a,b,c) in a_gender:
#     print(a+"-"+b)
we.viz(a_gender)

Computing neighbors
Mean: 996.986
Median: 998.0
   0                          she | he                           1.0
   1                          day | minutes                      0.29
   2                   tournament | minute                       0.26
   3                        court | members                      0.26
   4                      country | age                          0.26
   5                        serve | especially                   0.25
   6                      medical | situation                    0.25
   7                international | income                       0.25
   8                         stay | more                         0.24
   9                       series | charged                      0.24
  10                 applications | probably                     0.23
  11                         wife | performance                  0.23
  12                      planned | prices                       0.23
  13                      morning | most   

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [5]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.15426257, 'colonel'),
  (-0.15058891, 'councilor'),
  (-0.14500904, 'trooper'),
  (-0.14300711, 'physicist'),
  (-0.14258763, 'mediator'),
  (-0.14220968, 'cop'),
  (-0.14110851, 'surveyor'),
  (-0.14024815, 'undersecretary'),
  (-0.14019847, 'valedictorian'),
  (-0.13929208, 'maestro'),
  (-0.13917962, 'politician'),
  (-0.13780122, 'congressman'),
  (-0.13667953, 'narrator'),
  (-0.13652092, 'librarian'),
  (-0.1363942, 'vice_chancellor'),
  (-0.13622355, 'commander'),
  (-0.13523717, 'historian'),
  (-0.13507493, 'pharmacist'),
  (-0.13445523, 'administrator'),
  (-0.13370223, 'statesman')],
 [(-0.049532346, 'janitor'),
  (-0.049290616, 'bodyguard'),
  (-0.04897745, 'illustrator'),
  (-0.048656467, 'clerk'),
  (-0.047931194, 'fisherman'),
  (-0.046674065, 'secretary'),
  (-0.045202546, 'athletic_director'),
  (-0.041083053, 'evangelist'),
  (-0.040735602, 'correspondent'),
  (-0.039575815, 'solicitor_general'),
  (-0.039214253, 'financier'),
  (-0.03868881, 'preacher'),
  (-0.

## Part 2 Racial Bias

### Step 5: Define racial direction
We define racial direction based on the common names in different Demographic groups. 

In [6]:
names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
         "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
         "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
         "Leroy", "Brad", "Jermaine"]
names_group1 = [names[2 * i] for i in range(len(names) // 2)]
names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

In [7]:
# racial direction
vs = [sum(E.v(w) for w in names) for names in (names_group2, names_group1)]
vs = [v / np.linalg.norm(v) for v in vs]

v_racial = vs[1] - vs[0]
v_racial = v_racial / np.linalg.norm(v_racial)

### Step 6: Generating racial biased analogies

Similar to Step 3, we generate analogies that align with the racial dimension. 

In [11]:
# racial analogies
a_racial = E.best_analogies_dist_thresh(v_racial, max_words=1000)

# for (a,b,c) in a_racial:
#     print(a+"-"+b)
we.viz(a_racial)

   0                      players | talk                         0.29
   1                    announced | used                         0.29
   2                       ensure | very                         0.29
   3                     continue | offering                     0.27
   4                      details | access                       0.27
   5                   everything | only                         0.26
   6                        goals | use                          0.26
   7                     earnings | book                         0.25
   8                     provided | way                          0.25
   9                       scored | any                          0.24
  10                       within | order                        0.24
  11                      markets | built                        0.24
  12                      scoring | network                      0.24
  13                         news | lot                          0.24
  14                

### Step 7: Analyzing racial bias in word vectors asscoiated with professions

Similar to Step 4, we project occpurations onto the racial dimension.

In [9]:
# profession analysis racial
sp = sorted([(E.v(w).dot(v_racial), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.07526696, 'stockbroker'),
  (-0.072013855, 'shopkeeper'),
  (-0.06997903, 'publicist'),
  (-0.06425414, 'welder'),
  (-0.06420424, 'realtor'),
  (-0.06265733, 'epidemiologist'),
  (-0.061484993, 'bookkeeper'),
  (-0.059125546, 'housewife'),
  (-0.05522476, 'socialite'),
  (-0.049128495, 'baker'),
  (-0.048213784, 'cabbie'),
  (-0.04816783, 'organist'),
  (-0.04775571, 'drug_addict'),
  (-0.047214143, 'dermatologist'),
  (-0.045817573, 'sportswriter'),
  (-0.04415521, 'investment_banker'),
  (-0.04396136, 'environmentalist'),
  (-0.04165364, 'barber'),
  (-0.0414836, 'guidance_counselor'),
  (-0.041086137, 'pundit')],
 [(0.07394397, 'pianist'),
  (0.0743814, 'programmer'),
  (0.074969664, 'pathologist'),
  (0.07520779, 'correspondent'),
  (0.075593255, 'waiter'),
  (0.07573603, 'inventor'),
  (0.0758131, 'secretary'),
  (0.07964485, 'nun'),
  (0.08239908, 'pediatrician'),
  (0.087414026, 'chemist'),
  (0.0879491, 'footballer'),
  (0.095166184, 'inspector'),
  (0.10305147, 'comment

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [12]:
from debiaswe.debias import debias

In [13]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [14]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 768 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('SPOKESMAN', 'SPOKESWOMAN'), ('PROSTATE_CANCER', 'OVARIAN_CANCER'), ('MALE', 'FEMALE'), ('BOY', 'GIRL'), ('GRANDSONS', 'GRANDDAUGHTERS'), ('boy', 'girl'), ('male', 'female'), ('fathers', 'mothers'), ('CATHOLIC_PRIEST', 'NUN'), ('Dad', 'Mom'), ('MEN', 'WOMEN'), ('Gentleman', 'Lady'), ('dudes', 'gals'), ('Nephew', 'Niece'), ('father', 'mother'), ('DUDES', 'GALS'), ('BROTHERS', 'SISTERS'), ('GELDING', 'MARE'), ('Males', 'Females'), ('dad', 'mom'), ('grandpa', 'grandma'), ('BROTHER', 'SISTER'), ('prostate_cancer', 'ovarian_cancer'), ('fatherhood', 'motherhood'), ('HE', 'SHE'), ('son', 'daughter'), ('businessman', 'businesswoman'), ('wives', 'husbands'), ('SONS', 'DAUGHTERS'), ('Chairman', 'Chairwoman'), ('He', 'She'), ('CONGRESSMAN', 'CONGRESSWOMAN'), ('males', 'females'), ('nephew', 'niece'), ('brothers', 'sisters'), ('Uncle', 'Aunt'), ('Boys', 'Girls'), ('grandfather', 'grandmother'), ('Schoolboy', 'Schoo

In [15]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.16929759, 'councilor'),
  (-0.16828442, 'congressman'),
  (-0.16664192, 'colonel'),
  (-0.162978, 'mediator'),
  (-0.16251725, 'undersecretary'),
  (-0.16219766, 'trooper'),
  (-0.16036008, 'narrator'),
  (-0.15956642, 'politician'),
  (-0.15902817, 'physicist'),
  (-0.15899193, 'valedictorian'),
  (-0.157693, 'librarian'),
  (-0.15766348, 'administrator'),
  (-0.15666589, 'cop'),
  (-0.15642822, 'maestro'),
  (-0.15593147, 'commander'),
  (-0.15578131, 'surveyor'),
  (-0.15548019, 'vice_chancellor'),
  (-0.15481347, 'teenager'),
  (-0.15455693, 'neurosurgeon'),
  (-0.15375139, 'ambassador')],
 [(-0.06659895, 'clerk'),
  (-0.06503406, 'boxer'),
  (-0.06446973, 'tycoon'),
  (-0.062462583, 'athletic_director'),
  (-0.061547413, 'evangelist'),
  (-0.05618552, 'correspondent'),
  (-0.056026176, 'nun'),
  (-0.05575011, 'aide'),
  (-0.054137003, 'preacher'),
  (-0.051874723, 'solicitor_general'),
  (-0.051463068, 'accountant'),
  (-0.05085372, 'fisherman'),
  (-0.049876273, 'financier'

In [17]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender, max_words=1000)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 998.926
Median: 999.0
she-minutes
day-minute
women-between
woman-age
country-prices
wife-income
serve-especially
court-members
stay-probably
running-most
series-charged
medical-situation
remain-more
tournament-highest
morning-performance
afternoon-information
he-crisis
planned-only
show-time
international-member
world-all
like-senior
player-further
find-chance
applications-quite
prison-every
school-years
firm-however
come-won
her-us
children-child
equipment-longer
program-attention
policy-point
would-nothing
pay-getting
want-very
continues-likely
statements-provides
buy-sale
named-added
title-at
played-role
question-winner
served-limited
open-charge
filed-least
base-source
proposed-posted
food-less
summer-quickly
scheduled-position
website-personal
test-per
financial-products
died-can
track-failed
forces-few
old-funds
major-involved
field-points
kept-hold
does-do
department-something
hearing-meeting
playing-together
comment-per_cent
development-result
software